In [1]:
import dgl
import torch
from tqdm import tqdm
from time import process_time
import os
import numpy as np
import itertools
from torch.sparse import *
import collections
os.environ['DGLBACKEND'] = 'pytorch'

from scipy import stats

import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from ogb.nodeproppred import DglNodePropPredDataset, Evaluator

import matplotlib.pyplot as plt
import json
import pickle

This builds the 1-skeleton of a standard simplex

In [ ]:
class SimplexCreator():
    """Create standard simplex"""
    def __init__(self, dimension):
        self.input_dimension = dimension
        self.src=list()
        self.dst=list()
        for i in range(self.input_dimension+1):
            for j in range(self.input_dimension+1):
                if (i < j):
                    self.src = self.src + [i]
                    self.dst = self.dst + [j]

In [2]:
class psuedotv():
    src=list()
    dst=list()
    empty_graph = dgl.heterograph({('node', 'to', 'node'): (src, dst)})
    dimension = int

    assert isinstance(empty_graph, dgl.DGLHeteroGraph), \
        'Keyword argument \"graph\" of AdjGraph\'s init methodmust be a dgl.DGLHeteroGraph.'

    def __init__(
        self, file_path, graph=empty_graph,dimension=dimension):
        self.seed_graph = graph
        self.srcs_and_dsts = self.seed_graph.edges()   
        
        """Create dictionary with dimension (keys) and place list of nodes with all possible top dimensions (values)"""
        self.top_vertices_dictionary = collections.defaultdict(list)
        self.top_vertices_dictionary[0]=[int(x) for x in self.seed_graph.nodes()]
        self.top_vertices_dictionary[1]=[int(x) for x in torch.unique(self.srcs_and_dsts[1])]
        """ This dict has keys = dimensions d and values = dictionary. This subdictionary has keys = nodes and values =
        vertices that make the node a top-vertex of dimension d"""

        print("Finished adding 0-top vertices and 1-top vertices in main dictionary")
        
        """compute all in_degrees. These are needed for the algorithm later on when the loop runs"""
        self.in_degrees = self.seed_graph.in_degrees()
        self.out_degrees = self.seed_graph.out_degrees()
        self.dimension = dimension
        self.maximum_dimension = int(torch.max(self.in_degrees))
        
        self.file_path=file_path
        
        """Create dictionary with dimension (keys) and list of nodes with maximum
        dimension corresponding to key (values)"""
        self.pseudo_top_vertices_dict = collections.defaultdict(list)
        for v in self.seed_graph.nodes():
            """Add all zero-dimension vertices, and for now keep rest as 1-dimension vertices.
            The vertices moved to different keys as more simplices are identified"""
            if self.in_degrees[v] == 0:
                self.pseudo_top_vertices_dict[0] = self.pseudo_top_vertices_dict[0] + [int(v)]
            else:
                self.pseudo_top_vertices_dict[1] = self.pseudo_top_vertices_dict[1] + [int(v)]
                
        print("Finished adding 0-pseudo top vertices and 1-pseudo top vertices in main dictionary")
        
        """Create empty dictionary as above, but this time will have sets (value) for each dimension (key)"""        
        self.pseudo_top_vertices_dict_of_sets = dict()
        
        """Same values as above, but keys are addresses of the sets""" 
        self._sets = dict()
        
        """ Creates dictionary from above sets that has representatives (keys) and sets (values)"""
        self._partition = dict()

        """ Needed for refinement. Finds vertices in same class for each iteration of refinement"""
        self.refined_partition = dict()
        
        """ Number of refinements done after TV identification.for each vertex. Needed to find final partition index"""
        self.partition_number = 0
        
        """ Dictionary with keys = nodes as values = partition index of that node. Currently at zero, since
        no refinement is done, yet! The values for each key is supposed to be the partition_number"""
        self.partition_index = {int_node: 0 for int_node in self.top_vertices_dictionary[0]}
        
        """Create dictionary with keys = nodes and values = one-hot tensor of top-vertex and parition index,
        both concatenated"""
        self.partition_indices_and_one_hot_tv = {int_node: [] for int_node in self.top_vertices_dictionary[0]}
        
        """ Create dictionary with keys = nodes and values = one hot encoding of pseudo top dimension. That is
        ith index 1 if top maximum dimension is equal to index and zero otherwise """
        self.one_hot_dict = collections.defaultdict(list)

        """ Create dictionary with keys = nodes. The values for the dictionary 
        are tensors which are multi hot encoding with ith index 1 for i-top dimension and zero otherwise """
        self.multi_hot_tv_dict = collections.defaultdict(list)
        
        """ Create dictionary with keys = nodes and values = one hot encoding with ith index 1 if vertex is refined i
        times and zero otherwise. This is an index of number of times a vertex has been partitioned. """
        self.partition_times_hot_dict = collections.defaultdict(list)
        
        """ Create dictionary with keys = nodes and values = one hot encoding with ith index 1 if vertex is in the
        ith partition and zero otherwise. This captures the number of partitions after refinement. """
        self.partitioned_tv = collections.defaultdict(list)
        
        """ Create a dictionary of vectors R(v) for each vertex v. This gets updated at each refinement process. 
        The values are stored since then the algorithm wouldn't have to create the vector each time its needed"""
        self.refinement_vectors_dict = collections.defaultdict(list)
        
        """ Boolean expression to see if refinement needs to proceed"""
        self.partition_proceeds = True
        print("Computing matrices of dimension 2")
        diagonal_mask = (self.seed_graph.adj_external()._indices()[0] == self.seed_graph.adj_external()._indices()[1])
        off_diagonal_mask = ~diagonal_mask
        self.seed_graph.adj_external()._values()[off_diagonal_mask] = 1.0
        new_indices = self.seed_graph.adj_external()._indices()[:, off_diagonal_mask]
        new_values = self.seed_graph.adj_external()._values()[off_diagonal_mask]
        new_size = self.seed_graph.adj_external().size()
        new_sparse_matrix = torch.sparse_coo_tensor(indices=new_indices, values=new_values, size=new_size)
        self.hadamard_product_prev = new_sparse_matrix
        self.adj_matrix_without_diag = new_sparse_matrix
        self.powers_of_adj_matrix_witout_diag = new_sparse_matrix
        self.hadamard_product_next = torch.mul(new_sparse_matrix,torch.sparse.mm(new_sparse_matrix,new_sparse_matrix))
        print("Finished computing matrices for dimension 2")
        
    def __len__(self):
        """Return the size of the partition."""
        return len(self._sets)

    def out_nodes_as_int(self, vertex):
        """convert successors to a list with integer node values"""
        neighbors = [int(v) for v in list(self.seed_graph.successors(vertex))]
        if int(vertex) in neighbors:
            neighbors.remove(int(vertex))
        return neighbors

    def in_nodes_as_int(self, vertex):
        """convert predecessors to a list with integer node values"""
        neighbors = [int(v) for v in list(self.seed_graph.predecessors(vertex))]
        if int(vertex) in neighbors:
            neighbors.remove(int(vertex))
        return neighbors       
    
    def inductive_connecting(self):
        #Case for k=2 doesn't need to search for intersections:
        print("Adding 2-vertices to the dictionary")
        row_indices_analyzed_node, col_indices_analyzed_node = self.hadamard_product_next._indices()
        for vertex in col_indices_analyzed_node.unique():
            self.top_vertices_dictionary[2] = self.top_vertices_dictionary[2] + [int(vertex)]
            self.pseudo_top_vertices_dict[2] = self.pseudo_top_vertices_dict[2] + [int(vertex)]
            if vertex in self.pseudo_top_vertices_dict[1]:
                self.pseudo_top_vertices_dict[1].remove(int(vertex))
        print("Finished adding vertices of dimension 2")    
        
        for k in tqdm(range(3,self.dimension+1), position=0, leave=False):
            temp_power = torch.sparse.mm(self.powers_of_adj_matrix_witout_diag, self.adj_matrix_without_diag)
            diagonal_mask = (temp_power._indices()[0] == temp_power._indices()[1])
            off_diagonal_mask = ~diagonal_mask
            temp_power._values()[off_diagonal_mask] = 1.0
            new_indices = temp_power._indices()[:, off_diagonal_mask]
            new_values = temp_power._values()[off_diagonal_mask]
            new_size = temp_power.size()
            new_sparse_matrix = torch.sparse_coo_tensor(indices=new_indices, values=new_values, size=new_size)
            self.powers_of_adj_matrix_no_diag = new_sparse_matrix
            #compute A⚬A^2⚬..⚬A^k where ⚬ denotes Hadamard product
            self.hadamard_product_prev  = self.hadamard_product_next.clone().detach()
            self.hadamard_product_next = torch.mul(self.hadamard_product_prev,self.powers_of_adj_matrix_no_diag)
            print("Finished computing matrices for dim",k)
            
            for v in tqdm(self.seed_graph.nodes(), position=0, leave=False):
                if v in self.top_vertices_dictionary[k-1]:
                    if self.in_degrees[v] < k:
                        continue
                    else:
                        current_in_neighbors_of_node_analyzed = self.in_nodes_as_int(v)
                        for u in tqdm(self.top_vertices_dictionary[k-1], position=0, leave=False):
                            if u == v:
                                continue
                            if u not in current_in_neighbors_of_node_analyzed:
                                continue
                            else:
                                A = self.hadamard_product_prev
                                B = self.hadamard_product_next
                                A = A.coalesce()
                                B = B.coalesce()
                                # Get the row and column indices of the nonzero elements in each matrix
                                A_row, A_col = A.indices()
                                B_row, B_col = B.indices()
                                if len(B_col) == 0:
                                    print("We have reached maximum dimension, and that is",k-1)
                                    self.maximum_dimension = k-1
                                    self.top_vertices_dictionary.pop(k, None)
                                    return
                                # Find the rows that have a nonzero element in the u-th column of A
                                A_u_rows = A_row[torch.where(A_col.eq(u))]
                                # Find the rows that have a nonzero element in the v-th column of B
                                B_v_rows = B_row[torch.where(B_col.eq(v))]
                                # Find the common rows between A_u_rows and B_v_rows
                                common_rows = np.intersect1d(A_u_rows, B_v_rows)
                                #these are all the vertices for which there is a path with unique vertices
                                #of length 1, length 2, ..., length k-1 to both u and v
                                intersection_criterion = False
                                for i in common_rows:
                                    intersection = set(self.out_nodes_as_int(i)).intersection(
                                        set(self.in_nodes_as_int(u))).intersection(set(current_in_neighbors_of_node_analyzed))
                                    if len(intersection) > k-3:
                                        intersection_criterion = True
                                        break
                                if not(intersection_criterion):
                                    continue
                                else:
                                    self.top_vertices_dictionary[k] = self.top_vertices_dictionary[k] + [int(v)]
                                    self.pseudo_top_vertices_dict[k] = self.pseudo_top_vertices_dict[k] + [int(v)]
                                    if v in self.pseudo_top_vertices_dict[k-1]:
                                        self.pseudo_top_vertices_dict[k-1].remove(int(v))
                                        """a new top vertex v is found, so we can move out of the neighborhood of v"""
                                    break
                        
            if len(self.top_vertices_dictionary[k]) == 0:
                print("We have reached maximum dimension, and that is",k-1)
                self.maximum_dimension = k-1
                self.top_vertices_dictionary.pop(k, None)
                break
        
        print("Now creating other initial dictionaries")
        self.pseudo_top_vertices_dict_of_sets = {key:set(self.pseudo_top_vertices_dict[key]) 
                                                for key in range(0,self.maximum_dimension+1)}          
        self._sets = {id(self.pseudo_top_vertices_dict_of_sets[key]):self.pseudo_top_vertices_dict_of_sets[key]
                                 for key in self.pseudo_top_vertices_dict_of_sets.keys()}
        self._partition = {x:self.pseudo_top_vertices_dict_of_sets[key] 
                              for key in self.pseudo_top_vertices_dict_of_sets.keys() 
                           for x in self.pseudo_top_vertices_dict_of_sets[key]}
        print("Finished creating initial partition")
        print("Serializing dictionaries..")
        data_to_save = {"pseudo_top_vertices_dict": {str(key): value for key, value in self.pseudo_top_vertices_dict.items()}}
        
        with open('pseudo_tv.json', "w") as file:
            json.dump(data_to_save, file)
        print("Dictionary of pseudo top vertices saved as a JSON file")
    
    def partition_vector(self,vertex):
        """ Create vector R(v) for vertex v """ 
        vector = [self._partition[vertex]]
        for v in torch.sort(self.seed_graph.predecessors(vertex))[0]:
            """ The representatives have to be sorted for a meaningful comparison of vectors """
            if torch.eq(v,vertex):
                pass
            else:
                vector = vector + [self._partition[int(v)]]
        return vector
        
    def refine(self):
        """Original idea for refinement algorithm by David Eppstein. 
        Refine each set A in the partition to the two sets
        A & S, A - S.  Also produces output (A & S, A - S)
        for each changed set.  Within each pair, A & S will be
        a newly created set, while A - S will be a modified
        version of an existing set in the partition.
        """
        
        """ hit here is a dictionary with keys = addresses for original partitions and values = vertices with common
        partition vector"""
        hit = self.refined_partition
        output = list()
        for A,AS in hit.items():
            A = self._sets[A]
            """Check if new partition is not the same as the old partition"""
            if AS != A:
                self._sets[id(AS)] = AS
                """ This loop finds elements that are not part of previous partition"""
                for x in AS:
                    self._partition[x] = AS
                """The elements that were not part of the partition are now A"""
                A -= AS
                output = output + [set.union(A,AS)]
        """ output here keeps track of those partitions that have been broken down"""
        refined_vertices = set().union(*output)
        """The partitioning process above, once done, should then increase the partition_number, if the above
        does indeed count as another genuine refinement. If it does not, then the number of refined_vertices
        is zero, and hence should not increase partition number"""
        if len(refined_vertices) == 0:
            self.partition_proceeds = False
            return
        else:
            """If there is a refinement that takes place, then we increase the partition number and attach
            this as the partition index for each vertex in the partition_index dictionary"""
            self.partition_number = self.partition_number+1
            for v in refined_vertices:
                self.partition_index[v] = self.partition_number

    def refinement(self):
        """Keep on refining the partitions until the partition stabilizes"""
        while self.partition_proceeds:
            print("Refining..")
            """finds vertices u and v such that R(u) = R(v) and make refined partitions here"""
            common_vertices = dict()
            for node in tqdm(self.seed_graph.nodes(), position=0, leave=False):
                self.refinement_vectors_dict[int(node)] = self.partition_vector(int(node))
            print("Finished creating list of partition vectors for partition iteration number",self.partition_number)
            print("Finding vertices with common partition vectors..")
            """This step could be modified for optimization. It needlessly also checks
            for partion vectors of vertices that have not been partitioned the first time"""
            for v,u in tqdm(itertools.combinations(self.seed_graph.nodes(),2), position=0, leave=False):
                if self.refinement_vectors_dict[int(v)] == self.refinement_vectors_dict[int(u)]:
                    Au = self._partition[int(u)]
                    common_vertices.setdefault(id(Au),set()).update([int(u),int(v)])
            self.refined_partition=common_vertices  
            self.refine()
        print("Refinement process finished")

    def add_vertex_features(self):
        """First, we start with refinement until the partition stabilizes"""
        sets_for_partition_as_list = list(self._sets.values())
        print("Adding in node features..")
        for node in tqdm(self.seed_graph.nodes(), position=0, leave=False):
            """ Fills in the following dictionaries
            1) multi_hot_tv_dict
            2) one_hot_dict
            3) partition_indices_and_one_hot_tv
            4) partition_times_hot_dict
            5) partitioned_tv
            by filling in each key (node)"""
            pihvector = [0] * (self.partition_number+1)
            mhvector = [0] * (self.maximum_dimension+1)
            ohvector = [0] * (self.maximum_dimension+1)
            ptvvector = [0] * (len(self._sets))
            node = int(node)
            for dim in range(0,self.maximum_dimension+1):
                if node in self.top_vertices_dictionary[dim]:
                    mhvector[dim] = 1
                if node in self.pseudo_top_vertices_dict[dim]:
                    ohvector[dim] = 1
            pihvector[self.partition_index[node]] = 1
            self.multi_hot_tv_dict[node] = mhvector
            self.one_hot_dict[node] = ohvector
            self.partition_times_hot_dict[node] = pihvector
            self.partition_indices_and_one_hot_tv[node] = self.one_hot_dict[node] + self.partition_times_hot_dict[node]
            index = sets_for_partition_as_list.index(self._partition[node])
            ptvvector[index] = 1
            self.partitioned_tv[node] = ptvvector
        print("Vertex features added!")
          
    def save_dicts(self):
        print("Serializing dictionaries for vertex features..")
        data_to_save = {
            "partitioned_tv": {str(key): value for key, value in self.partitioned_tv.items()},
            "partition_indices_and_one_hot_tv": {str(key): value for key, value in self.partition_indices_and_one_hot_tv.items()},
            "multi_hot_tv_dict": {str(key): value for key, value in self.multi_hot_tv_dict.items()},
            "partition_times_hot_dict" : {str(key): value for key, value in self.partition_times_hot_dict.items()}
        }
        
        with open(self.file_path, "w") as file:
            json.dump(data_to_save, file)
        print("Dictionaries saved as a JSON file")
        
    def load_dicts(self):
        with open(self.file_path, "r") as file:
            loaded_data = json.load(file)
        
        self.partitioned_tv = {int(key): torch.tensor(value) for key, value in loaded_data["partitioned_tv"].items()}
        self.partition_indices_and_one_hot_tv = {int(key): torch.tensor(value) for key, value in loaded_data["partition_indices_and_one_hot_tv"].items()}
        self.multi_hot_tv_dict = {int(key): torch.tensor(value) for key, value in loaded_data["multi_hot_tv_dict"].items()}
        self.partition_times_hot_dict = {int(key): torch.tensor(value) for key, value in loaded_data["partition_times_hot_dict"].items()}
        
    def load_ptv_dict(self,path):
        with open(path, "r") as file:
            loaded_data = json.load(file)        
        self.pseudo_top_vertices_dict = {int(key): torch.tensor(value) for key, value in loaded_data["pseudo_top_vertices_dict"].items()}
        for key in range(self.dimension, len(self.seed_graph.nodes())):
            if len(self.pseudo_top_vertices_dict[key]==0):
                del self.pseudo_top_vertices_dict[key]

    def make_plots(self, dict_name):
        d = self.maximum_dimension    
        # x axis values 
        x = range(0, d+1)
        print("d=",d)
        # corresponding y axis values
        if dict_name not in ['partitioned_tv','tv']:
            raise ValueError("Invalid dictionary name. Must be either partitioned_tv or tv")
        y = list()
        if dict_name == 'partitioned_tv':
            for key in self._sets.keys():
                y.append(len(self._sets[key]))
            plt.figure(figsize=(10, 6))
            plt.bar(x, y, color='blue') 
            plt.xlabel('dimension') 
            plt.xlabel('Number of elements in partition')
            plt.title('representative') 
            plt.xticks(x)
            plt.grid(True)
            plt.tight_layout()
            plt.show()
            plt.savefig('partitionedtv_plot.png')
        if dict_name == 'tv':
            for key in self.top_vertices_dictionary.keys():
                y.append(len(self.top_vertices_dictionary[key]))
            plt.figure(figsize=(10, 6))
            plt.bar(x, y, color='blue') 
            plt.xlabel('dimension') 
            plt.xlabel('Number of partitioned top vertices')
            plt.title('dimension distribution') 
            plt.xticks(x)
            plt.grid(True)
            plt.tight_layout()
            plt.show()
            plt.savefig('partitionedtv_plot.png')          

class PartitionError(Exception): pass

In [ ]:
"""Code testing"""
K_10 = dgl.heterograph({('paper', 'cites', 'paper'): (SimplexCreator(dimension=20).src, SimplexCreator(dimension=20).dst)})
filepath = 'K_10'
K_10_preprocessing = psuedotv(filepath,graph=K_10,dimension=30)
K_10_preprocessing.inductive_connecting()
print("Top vertices dictionary=",K_10_preprocessing.top_vertices_dictionary)
print("Partition by dimension=",K_10_preprocessing.pure_top_vertices_dict)
print("top vertex dictionary=",K_10_preprocessing.top_vertices_dictionary)
print("Partitions before refinement=",K_10_preprocessing._partition)
K_10_preprocessing.refinement()
K_10_preprocessing.add_vertex_features()
print("Partitions after refinement=",K_10_preprocessing._partition)
print("One hot encoding of pure tv=",K_10_preprocessing.one_hot_dict)
print("multi hot encoding of all dimensions for tv=",K_10_preprocessing.multi_hot_tv_dict)
print("partition indices=",K_10_preprocessing.partition_index)
print("tv + partition index hot dict=",K_10_preprocessing.partition_indices_and_one_hot_tv)
print("partitioned one-hot encoding, with index 1 if vertex is refined ith time=",K_10_preprocessing.partitioned_tv)
print("index of partitions=",K_10_preprocessing.partition_times_hot_dict)

In [3]:
dataset = DglNodePropPredDataset(name = "ogbn-arxiv", root = 'dataset/')
arxiv_graph = dataset.graph[0]
filepath = 'arxiv'
arxiv_preprocessing = psuedotv(filepath,graph=arxiv_graph,dimension=30)
arxiv_preprocessing.load_ptv_dict('semi_tv_upto5.json')

Finished adding 0-top vertices and 1-top vertices in main dictionary
Finished adding 0-pseudo top vertices and 1-pseudo top vertices in main dictionary
Computing matrices of dimension 2
Finished computing matrices for dimension 2


KeyError: 'pseudo_top_vertices_dict'

In [ ]:
"""Code testing"""
src_absurd_case4  = [0,0,0,1,1,2] + [1] + [4, 4, 4, 5, 5, 6] 
dst_absured_case4 = [1,2,3,2,3,3] + [4] + [5, 6, 7, 6, 7, 7]
absured_case4 = dgl.heterograph({('paper', 'cites', 'paper'): (src_absurd_case4, dst_absured_case4)})
filepath = 'troubling_graph'
troubling_graph_preprocessing = psuedotv(filepath,graph=absured_case4,dimension=30)
troubling_graph_preprocessing.inductive_connecting() 
print("Top vertices dictionary=",troubling_graph_preprocessing.top_vertices_dictionary)
print("Partition by dimension=",troubling_graph_preprocessing.pure_top_vertices_dict)
print("top vertex dictionary=",troubling_graph_preprocessing.top_vertices_dictionary)
print("Partitions before refinement=",troubling_graph_preprocessing._partition)
troubling_graph_preprocessing.refinement()
troubling_graph_preprocessing.add_vertex_features()
print("Partitions after refinement=",troubling_graph_preprocessing._partition)
print("One hot encoding of pure tv=",troubling_graph_preprocessing.one_hot_dict)
print("multi hot encoding of all dimensions for tv=",troubling_graph_preprocessing.multi_hot_tv_dict)
print("partition indices=",troubling_graph_preprocessing.partition_index)
print("tv + partition index hot dict=",troubling_graph_preprocessing.partition_indices_and_one_hot_tv)
print("partitioned one-hot encoding, with index 1 if vertex is refined ith time=",troubling_graph_preprocessing.partitioned_tv)
print("index of partitions=",troubling_graph_preprocessing.partition_times_hot_dict)

In [ ]:
import random
def generate_random_graph(num_nodes):
    src_edges =[]
    dst_edges = []
    edges = []
    for i in range(2*num_nodes):
        src_edges.append(random.randint(0,num_nodes))
        dst_edges.append(random.randint(0,num_nodes))
        edges.append((src_edges[i],dst_edges[i]))
    graph = dgl.heterograph({('paper', 'cites', 'paper'): (src_edges, dst_edges)})
    return graph, edges

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

dgl_G, edges = generate_random_graph(10)
print(edges)
nx_G = nx.DiGraph()
nx_G.add_edges_from(edges)
options = {
    'node_color': 'black',
    'node_size': 20,
    'width': 1,
}
#pos = nx.spring_layout(nx_G, seed=42)
pos = nx.planar_layout(nx_G)
nx.draw_networkx(nx_G, pos, with_labels=True, node_color='lightblue', node_size=200, font_size=10, font_color='black', arrows=True)
plt.show()

In [ ]:
filepath='randomgraph'
random_graph_preprocessing = psuedotv(filepath,graph=dgl_G,dimension=30)
random_graph_preprocessing.inductive_connecting()
print("Top vertices dictionary=",random_graph_preprocessing.top_vertices_dictionary)
print("Partition by dimension=",random_graph_preprocessing.pure_top_vertices_dict)
print("top vertex dictionary=",random_graph_preprocessing.top_vertices_dictionary)
print("Partitions before refinement=",random_graph_preprocessing._partition)
random_graph_preprocessing.refinement()
random_graph_preprocessing.add_vertex_features()
print("Partitions after refinement=",random_graph_preprocessing._partition)
print("One hot encoding of pure tv=",random_graph_preprocessing.one_hot_dict)
print("multi hot encoding of all dimensions for tv=",random_graph_preprocessing.multi_hot_tv_dict)
print("partition indices=",random_graph_preprocessing.partition_index)
print("tv + partition index hot dict=",random_graph_preprocessing.partition_indices_and_one_hot_tv)
print("partitioned one-hot encoding, with index 1 if vertex is refined ith time=",random_graph_preprocessing.partitioned_tv)
print("index of partitions=",random_graph_preprocessing.partition_times_hot_dict)